In [1]:
from itertools import zip_longest
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np
import csv
import pandas as pd
import numpy as np
import time
start_time = time.time()

In [2]:
Doc = pd.read_csv('BTCUSDT-1m-data.csv',encoding = 'utf-8') 

,timestamp,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore
0,2017-08-17 04:00:00,4261.48,4261.48,4261.48,4261.48,1.775183,1502942459999,7564.906851,3,0.075183,320.390851,7960.540180
1,2017-08-17 04:01:00,4261.48,4261.48,4261.48,4261.48,0.000000,1502942519999,0.000000,0,0.000000,0.000000,7960.452630
2,2017-08-17 04:02:00,4280.56,4280.56,4280.56,4280.56,0.261074,1502942579999,1117.542921,2,0.261074,1117.542921,7960.375295


In [3]:
Doc = Doc.drop(Doc.columns[[1,2,3,5,6,7,8,9,10,11]], axis=1)  
Doc.close= pd.to_numeric(Doc.close) # converts close_price to numeric value for plotting
Doc.head(3)

,timestamp,close
0,2017-08-17 04:00:00,4261.48
1,2017-08-17 04:01:00,4261.48
2,2017-08-17 04:02:00,4280.56


In [4]:
# Define the Sample data:

sample_start = 595156
sample_finish = 596016
sample_length= (sample_finish-sample_start)

#  TODAY'S FORECAST:
# 
#present_time = time_cleaned[-1] 
#a_day_before_present_time = present_time - (1*60*24)
#sample_start = a_day_before_present_time
#sample_finish = present_time
#sample_length= (sample_finish-sample_start)
# Sample prices are given to us as strings but we want float so we must clean data:
sample_prices = Doc["close"][sample_start:sample_finish]
sample_length

860

In [5]:
Doc[596015:596020]

,timestamp,close
596015,2018-10-07 17:37:00,6595.79
596016,2018-10-07 17:38:00,6594.77
596017,2018-10-07 17:39:00,6592.06
596018,2018-10-07 17:40:00,6593.49
596019,2018-10-07 17:41:00,6590.01


In [6]:
len(sample_prices)

860

In [7]:
# PERFORMING CORRELATION CALCULATION ITERATION with our sample_prices

Doc["close"]
window_size = sample_length

i = 0
moving_correlation=[]
while i <len(Doc["close"]) -window_size +1:
    this_window = Doc["close"][i : i + window_size]
    
    window_correlation= correlation, p_value = stats.pearsonr(this_window, sample_prices)
    moving_correlation.append(window_correlation)
    i += 1
    
# output (moving_correlation) appears to be a list of tuples?? with 2 elements inside the tuple? I was expecting one list of single floats??
# after checking the numbers I found that we only need the 1st number as it matches with the csv figures!

In [8]:
correlations = []
for item in moving_correlation:
    correlations.append(item[0])
# we have extracted the first number in the tuples to lists, idk what the 2nd number is.

In [9]:
empty_list =[0] * (sample_length-1) # creates empty list as the window_correlation is zero.
empty_list.extend(correlations) # adds the correlation values onto the end of the empty list
Doc["Correlation"]= empty_list # empty isnt empty anymore is it? Now pd is full with what we need
Doc_sort_by_correlation = Doc.sort_values(['Correlation'], ascending=[False]) # Gives us top pattern matches 
Doc_sort_by_correlation[0:3]

,timestamp,close,Correlation
596015,2018-10-07 17:37:00,6595.79,1.000000
596014,2018-10-07 17:36:00,6593.01,0.994304
596016,2018-10-07 17:38:00,6594.77,0.994297


In [10]:
Doc[0:4]

,timestamp,close,Correlation
0,2017-08-17 04:00:00,4261.48,0.0
1,2017-08-17 04:01:00,4261.48,0.0
2,2017-08-17 04:02:00,4280.56,0.0
3,2017-08-17 04:03:00,4261.48,0.0


In [11]:
# PURGE! IMPRESSIVE!, this code removes all the overlapping, takes value of list look left by increment, looks right by increment
# purges all those left and right, looks at the next and does the same, remember that this works because our list has already
# been sorted from when we created the time_and_correlation_dictionary_sorted
# was the only way we could delete/purge/remove overlaps since we could not operate on a dictionary, cannot loop on a dictionary
# 1 filteredtime
increment = sample_length

to_remove = set()
for x in Doc_sort_by_correlation.index.tolist():
    if x not in to_remove:
        # Number is not going to be removed by previous, so it's going to be
        # the next purge candidate.
        for i in range(1, 1 + increment):
            to_remove.add(x - i)
            to_remove.add(x + i)

filtered_time = [x for x in Doc_sort_by_correlation.index.tolist() if x not in to_remove]
filtered_time[0:4]

[596015, 2013214, 363197, 584138]

In [12]:
patterns_df =pd.DataFrame(index =filtered_time)
patterns_df = pd.merge(patterns_df, Doc,left_index=True,right_index=True) # inner merge, removes all other values from first df which do not have a corresponding time value for the 2nd df
patterns_df

,timestamp,close,Correlation
596015,2018-10-07 17:37:00,6595.79,1.000000
2013214,2021-06-20 19:54:00,35394.07,0.923738
363197,2018-04-28 05:57:00,9158.73,0.919525
584138,2018-09-29 11:40:00,6597.87,0.913243
233492,2018-01-26 17:53:00,11172.00,0.910788
...,...,...,...
1902955,2021-04-04 23:32:00,58178.32,-0.713434
1519576,2020-07-12 09:14:00,9245.96,-0.721403
889,2017-08-17 18:49:00,4241.90,-0.725289
1909354,2021-04-09 10:11:00,58225.30,-0.747117


In [29]:
#INPUTS! INPUTS! INPUTS! INPUTS! INPUTS! INPUTS! INPUTS! INPUTS! INPUTS! INPUTS! INPUTS! INPUTS! INPUTS! INPUTS! 
######################################################################################################################
berry = 8
leap = 0 *berry


dataset1end   =      filtered_time[1+leap]                    
dataset1start =  dataset1end - sample_length   

dataset2end   =      filtered_time[2+leap]                     
dataset2start =  dataset2end - sample_length   
                                               
dataset3end   =      filtered_time[3+leap]               
dataset3start =  dataset3end - sample_length

dataset4end   =      filtered_time[4+leap]               
dataset4start=  dataset4end - sample_length

dataset5end   =       filtered_time[5+leap]               
dataset5start =  dataset5end - sample_length

dataset6end   =      filtered_time[6+leap]              
dataset6start =  dataset6end - sample_length

dataset7end   =       filtered_time[7+leap]            
dataset7start =  dataset7end - sample_length

dataset8end   =       filtered_time[8+leap]              
dataset8start =  dataset8end - sample_length


future_projection = 700                          #######################################################################
                                               #######################################################################  
######################################################################################################################
X = []
Y = []
A = []
B = []
C = []
D = []
E = []
F = []
G = []
H = []
I = []
J = []
K = []
L = []
M = []
N = []
O = []
P = []

# List Loops for Plotting     
#########################################################################################################################

# For our Sample Data (sample):    
for element in Doc.index.tolist()[sample_start:sample_finish]:
    X.append(element)    
for element in sample_prices:
    Y.append(element)
    
# For our Data Set 1 (large_data_set)
for element in Doc.index.tolist()[dataset1start:dataset1end + future_projection]:
    A.append(element)
for element in Doc["close"][dataset1start:dataset1end + future_projection]:
    B.append(element)
    
# For our Data Set 2 (large_data_set):       
for element in Doc.index.tolist()[dataset2start:dataset2end + future_projection]:
    C.append(element)
for element in Doc["close"][dataset2start:dataset2end + future_projection]:
    D.append(element)
    
# For our Data Set 3 (large_data_set):       
for element in Doc.index.tolist()[dataset3start:dataset3end + future_projection]:
    E.append(element)
for element in Doc["close"][dataset3start:dataset3end + future_projection]:
    F.append(element)
    
    #######################################
    
for element in Doc.index.tolist()[dataset4start:dataset4end + future_projection]:
    G.append(element)
for element in Doc["close"][dataset4start:dataset4end + future_projection]:
    H.append(element)

for element in Doc.index.tolist()[dataset5start:dataset5end + future_projection]:
    I.append(element)
for element in Doc["close"][dataset5start:dataset5end + future_projection]:
    J.append(element)
    
for element in Doc.index.tolist()[dataset6start:dataset6end + future_projection]:
    K.append(element)
for element in Doc["close"][dataset6start:dataset6end + future_projection]:
    L.append(element)
    
for element in Doc.index.tolist()[dataset7start:dataset7end + future_projection]:
    M.append(element)
for element in Doc["close"][dataset7start:dataset7end + future_projection]:
    N.append(element)
    
for element in Doc.index.tolist()[dataset8start:dataset8end + future_projection]:
    O.append(element)
for element in Doc["close"][dataset8start:dataset8end + future_projection]:
    P.append(element)

In [30]:
from bokeh.layouts import gridplot
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.models import FreehandDrawTool

output_file("October_Matches.html")

TOOLS = "crosshair,pan,wheel_zoom,box_zoom,reset,box_select,lasso_select,poly_select,save,undo"

# create a new plot

Sample_Data = figure(tools=TOOLS, width=600, x_axis_label='TIME', y_axis_label='PRICE',  plot_height=400, title= f"SAMPLE DATA Correlation")
Sample_Data.line(X, Y, line_width=3, color="black", alpha=1)
Sample_Data.ygrid.grid_line_color = None
Sample_Data.ygrid.grid_line_color = None

ticks = [sample_finish]
Sample_Data.xaxis[0].ticker = ticks
Sample_Data.xgrid[0].ticker = ticks
Sample_Data.xgrid.band_fill_alpha  = 0.1
Sample_Data.xgrid.band_fill_color = "navy"


renderer = Sample_Data.multi_line([[sample_start, sample_start+1]], [[Y[0], Y[1]]], line_width=5, alpha=0.4, color='red')
draw_tool = FreehandDrawTool(renderers=[renderer], num_objects=10)
Sample_Data.add_tools(draw_tool)
Sample_Data.toolbar.active_drag = draw_tool


Data_Set_1 = figure(tools=TOOLS, width=600, x_axis_label='TIME', y_axis_label='PRICE',   height=400, title= f"Data Set 1 Correlation")
Data_Set_1.line(A, B, line_width=1, color="navy", alpha=1)
Data_Set_1.ygrid.grid_line_color = None
Data_Set_1.ygrid.grid_line_color = None
ticks = [0,1, dataset1end]
Data_Set_1.xaxis[0].ticker = ticks
Data_Set_1.xgrid[0].ticker = ticks
Data_Set_1.xgrid.band_fill_alpha  = 0.1
Data_Set_1.xgrid.band_fill_color = "navy"

renderer = Data_Set_1.multi_line([[dataset1start, dataset1start+1]], [[B[0], B[1]]], line_width=5, alpha=0.4, color='red')
draw_tool = FreehandDrawTool(renderers=[renderer], num_objects=10)
Data_Set_1.add_tools(draw_tool)
Data_Set_1.toolbar.active_drag = draw_tool

Data_Set_2 = figure(tools=TOOLS, width=600, x_axis_label='TIME', y_axis_label='PRICE',  height=400, title= f"Data Set 2 ")
Data_Set_2.line(C, D, line_width=1, color="deeppink", alpha=1)
ticks = [0,1, dataset2end]
Data_Set_2.xaxis[0].ticker = ticks
Data_Set_2.xgrid[0].ticker = ticks
Data_Set_2.xgrid.band_fill_alpha  = 0.1
Data_Set_2.xgrid.band_fill_color = "navy"

renderer = Data_Set_2.multi_line([[dataset2start, dataset2start+1]], [[D[0], D[1]]], line_width=5, alpha=0.4, color='red')
draw_tool = FreehandDrawTool(renderers=[renderer], num_objects=10)
Data_Set_2.add_tools(draw_tool)
Data_Set_2.toolbar.active_drag = draw_tool

Data_Set_3 = figure(tools=TOOLS, width=600, x_axis_label='TIME', y_axis_label='PRICE',  height=400, title= f"Data Set 3 ")
Data_Set_3.line(E, F, line_width=1, color="green", alpha=1)
ticks = [0,1, dataset3end]
Data_Set_3.xaxis[0].ticker = ticks
Data_Set_3.xgrid[0].ticker = ticks
Data_Set_3.xgrid.band_fill_alpha  = 0.1
Data_Set_3.xgrid.band_fill_color = "navy"

renderer = Data_Set_3.multi_line([[dataset3start, dataset3start+1]], [[F[0], F[1]]], line_width=5, alpha=0.4, color='red')
draw_tool = FreehandDrawTool(renderers=[renderer], num_objects=10)
Data_Set_3.add_tools(draw_tool)
Data_Set_3.toolbar.active_drag = draw_tool


Data_Set_4 = figure(tools=TOOLS, width=600, x_axis_label='TIME', y_axis_label='PRICE',  height=400, title= f"Data Set 4")
Data_Set_4.line(G, H, line_width=1, color="firebrick", alpha=1)
ticks = [0,1, dataset4end]
Data_Set_4.xaxis[0].ticker = ticks
Data_Set_4.xgrid[0].ticker = ticks
Data_Set_4.xgrid.band_fill_alpha  = 0.1
Data_Set_4.xgrid.band_fill_color = "navy"

renderer = Data_Set_4.multi_line([[dataset4start, dataset4start+1]], [[H[0], H[1]]], line_width=5, alpha=0.4, color='red')
draw_tool = FreehandDrawTool(renderers=[renderer], num_objects=10)
Data_Set_4.add_tools(draw_tool)
Data_Set_4.toolbar.active_drag = draw_tool


Data_Set_5 = figure(tools=TOOLS, width=600, x_axis_label='TIME', y_axis_label='PRICE', height=400, title= f"Data Set 5")
Data_Set_5.line(I, J, line_width=1, color="darkorange", alpha=1)
ticks = [0,1, dataset5end]
Data_Set_5.xaxis[0].ticker = ticks
Data_Set_5.xgrid[0].ticker = ticks
Data_Set_5.xgrid.band_fill_alpha  = 0.1
Data_Set_5.xgrid.band_fill_color = "navy"

renderer = Data_Set_5.multi_line([[dataset5start, dataset5start+1]], [[J[0], J[1]]], line_width=5, alpha=0.4, color='red')
draw_tool = FreehandDrawTool(renderers=[renderer], num_objects=10)
Data_Set_5.add_tools(draw_tool)
Data_Set_5.toolbar.active_drag = draw_tool

Data_Set_6 = figure(tools=TOOLS, width=600, x_axis_label='TIME', y_axis_label='PRICE',  height=400, title= f"Data Set 6")
Data_Set_6.line(K, L, line_width=1, color="darkviolet", alpha=1)
ticks = [0,1, dataset6end]
Data_Set_6.xaxis[0].ticker = ticks
Data_Set_6.xgrid[0].ticker = ticks
Data_Set_6.xgrid.band_fill_alpha  = 0.1
Data_Set_6.xgrid.band_fill_color = "navy"

renderer = Data_Set_6.multi_line([[dataset6start, dataset6start+1]], [[L[0], L[1]]], line_width=5, alpha=0.4, color='red')
draw_tool = FreehandDrawTool(renderers=[renderer], num_objects=10)
Data_Set_6.add_tools(draw_tool)
Data_Set_6.toolbar.active_drag = draw_tool

Data_Set_7 = figure(tools=TOOLS, width=600, x_axis_label='TIME', y_axis_label='PRICE', height=400, title= f"Data Set 7")
Data_Set_7.line(M, N, line_width=1, color="blue", alpha=1)
ticks = [0,1, dataset7end]
Data_Set_7.xaxis[0].ticker = ticks
Data_Set_7.xgrid[0].ticker = ticks
Data_Set_7.xgrid.band_fill_alpha  = 0.1
Data_Set_7.xgrid.band_fill_color = "navy"

renderer = Data_Set_7.multi_line([[dataset7start, dataset7start+1]], [[N[0], N[1]]], line_width=5, alpha=0.4, color='red')
draw_tool = FreehandDrawTool(renderers=[renderer], num_objects=10)
Data_Set_7.add_tools(draw_tool)
Data_Set_7.toolbar.active_drag = draw_tool

Data_Set_8 = figure(tools=TOOLS, width=600, x_axis_label='TIME', y_axis_label='PRICE', height=400, title= f"Data Set 8")
Data_Set_8.line(O, P, line_width=1, color="red", alpha=1)
ticks = [0,1, dataset8end]
Data_Set_8.xaxis[0].ticker = ticks
Data_Set_8.xgrid[0].ticker = ticks
Data_Set_8.xgrid.band_fill_alpha  = 0.1
Data_Set_8.xgrid.band_fill_color = "navy"

renderer = Data_Set_8.multi_line([[dataset8start, dataset8start+1]], [[P[0], P[1]]], line_width=5, alpha=0.4, color='red')
draw_tool = FreehandDrawTool(renderers=[renderer], num_objects=10)
Data_Set_8.add_tools(draw_tool)
Data_Set_8.toolbar.active_drag = draw_tool

# put all the plots in a grid layout
p = gridplot( [[Sample_Data, Data_Set_1, Data_Set_2], [Data_Set_3, Data_Set_4, Data_Set_5], [Data_Set_6, Data_Set_7, Data_Set_8]] )

# show the results
show(p)

In [14]:
runtime = time.time() - start_time
runtime_whole_minutes = round(runtime/60)
runtime_leftover_minutes= runtime % 60

print (f"My program took, {runtime} seconds, to run")
print (f"which is, {runtime_whole_minutes}:{runtime_leftover_minutes} minutes, to run")

My program took, 251.6696960926056 seconds, to run
which is, 4:11.66969609260559 minutes, to run
